In [1]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd

# Carga tu dataframe (aquí asumo que el archivo está en formato CSV)
df = pd.read_csv("imdb_ds_2k_clean.csv")  # Cambia esto por la ruta a tu archivo
df

2025-09-11 00:10:25.273989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757567425.314197 3528186 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757567425.326916 3528186 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-11 00:10:25.420367: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,sw_text,sentiment
0,one reviewer ha mention watch oz episode youll...,positive
1,wonderful little production film technique una...,positive
2,think wa wonderful way spend time hot summer w...,positive
3,basically family little boy jake think zombie ...,negative
4,petter matteis love time money visually stun f...,positive
...,...,...
1995,feel minnesota direct steven baigelmann star k...,negative
1996,cell rat cell like antz must watch twice appre...,positive
1997,movie despite list list celebs complete waste ...,negative
1998,love movie wa could break tear watch really up...,positive


In [2]:
reviews = df["sw_text"].tolist()  # Extrae la columna de reviews como lista de oraciones

In [3]:
# Inicializa el tokenizador y modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [4]:
from datetime import datetime
import time
# Función para generar embeddings de una palabra
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Promedia los embeddings para los tokens de la palabra
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    # Reduce la dimensión de 768 a 300
    embeddings = embeddings[:300] if embeddings.shape[0] > 300 else np.pad(embeddings, (0, 300 - embeddings.shape[0]), 'constant')
    return embeddings


tiempo_inicio = time.time()

# Prepara un conjunto único de palabras en todas las reviews
unique_words = set()
for review in reviews:
    words = review.split()
    unique_words.update(words)

# Abre un archivo para escribir los embeddings
with open("embedding_imdb2k_bert_092025.txt", "w") as f:
    # Escribe el encabezado con el número de palabras y la dimensión
    f.write(f"{len(unique_words)} 300\n")
    
    for word in unique_words:
        # Obtén el embedding de la palabra
        embedding = get_word_embedding(word)
        # Convierte el embedding en una cadena de texto
        embedding_str = " ".join(map(str, embedding))
        # Escribe la palabra seguida de su embedding
        f.write(f"{word} {embedding_str}\n")

print("Embeddings guardados en 'word_embeddings.txt'") #5min

tiempo_final = time.time()
duracion = tiempo_final - tiempo_inicio  # segundos

horas = int(duracion // 3600)
minutos = int((duracion % 3600) // 60)
segundos = int(duracion % 60)

print(f"Tiempo de ejecución vectorización BERT: {horas} h {minutos} min {segundos} s "f"({duracion/3600:.4f} horas)")


tiempo_final2 = time.time()
tiempo_ejecucion_horas = (tiempo_final2 - tiempo_inicio) / 3600
print(f"Tiempo de ejecución vectorizacion de palabras BERT: {tiempo_ejecucion_horas:.4f} horas")


Embeddings guardados en 'word_embeddings.txt'
Tiempo de ejecución vectorización BERT: 0 h 6 min 5 s (0.1016 horas)
Tiempo de ejecución vectorizacion de palabras BERT: 0.1016 horas
